## 0. Introduction

The aim of this lab is to get familiar with **Age Estimation** from **facial images** using  **regression**. In particular, you are required to understand and build programs that take the
AAM parameters as representation of human faces and learn a regression function to predict age for an unseen face.


1.   This lab is the third course-work activity **Assignment 3**
2.   A report answering the <font color = 'red'>**questions in</font><font color = "maroon"> red**</font> should be submitted on QMplus along with the completed Notebook.
3. The report should be a separate file in **pdf format** (so **NOT** *doc, docx, notebook* etc.), well identified with your name, student number, assignment number (for instance, Assignment 1), module code.
4. Make sure that **any figures or code** you comment on, are **included in the report**.
5. No other means of submission other than the appropriate QM+ link is acceptable at any time (so NO email attachments, etc.)
6. **PLAGIARISM** <ins>is an irreversible non-negotiable failure in the course</ins> (if in doubt of what constitutes plagiarism, ask!).

In [ ]:
import torch
from torch import nn
from torch import optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import numpy as np

from sklearn.cross_decomposition import PLSRegression


We will use the [FGNET dataset](https://yanweifu.github.io/FG_NET_data/) for this lab.It is a dataset for age estimation and face recognition across ages. It is composed of a total of 1,002 images with 82 people aged 0 to 69. It is often used for face verification across large age gaps. The dataset contains images ranging from child/young to adult/old.


The `.npz` files `data_age` and `data_age_test` respectively, contain the precomputed AAM features for the dataset.


In [ ]:
train_file = 'data_age.npz'
test_file = 'data_age_test.npz'
train = np.load(train_file)
test = np.load(test_file)

In [ ]:
class FGNET(Dataset):
    def __init__(self, data, labels):
        super().__init__()
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

train_data = FGNET(train['features'], train['labels'])
test_data = FGNET(test['features'], test['labels'])


train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64)

# 1. Linear Regression

We will use pytorch [Linear](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html) layer and Stochastic Gradient Descent [(SGD)](https://pytorch.org/docs/stable/generated/torch.optim.SGD.html?highlight=sgd) to do the age estimation. We will use the Mean Absolute Error [(MAE)](https://pytorch.org/docs/stable/generated/torch.nn.L1Loss.html?highlight=mae)

Task 1: Train a linear regression model using pytorch on Age Estimation [4 marks]

In [ ]:
### YOUR CODE HERE
# Step 1: Create a sequential model with a single nn.Linear layer.
# Set the number of inputs and outputs according to the dimensionality of
# the features and labels accordingly
sample_features, _ = train_data[0]
n = sample_features.shape[0]

# model = nn.Sequential("linear regression layer")
model = nn.Sequential(
    nn.Linear(in_features=n, out_features=1)
)

# Step 2: Create an SGD optimizer
# Hint: check pytorch API

# optimizer = 0
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Step 3: Create the mae criterion

# criterion = 0
criterion = nn.L1Loss()

# Step 4: Train the model
for epoch in range(50):
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs = inputs.float()
        targets = targets.float()
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

Task 2: Apply the learned linear regressionmodel to estimate the age for each test data point. Compute the MAE and CS value (with a cumulative error level of 5) by comparing the estimated ages with the ground truth ages. [4 marks]

In [ ]:
### YOUR CODE HERE
model.eval()

total_mae = 0
total_cs = 0
mae_criterion = nn.L1Loss(reduction='sum')

with torch.no_grad():
    for data, labels in test_loader:
        data = data.float()
        labels = labels.float()
        predictions = model(data)
        total_mae += mae_criterion(predictions, labels).item()
        cs=(torch.abs(predictions - labels) <= 5).float().mean().item()
        total_cs += cs * data.size(0)

average_mae = total_mae / len(test_loader.dataset)
cs_value = total_cs / len(test_loader.dataset)

print(f"linear regression Average MAE: {average_mae}")
print(f"linear regression CS (Error <= 5): {cs_value}")

Task 3: <font color = 'red'>  Vary the cumulative error level from 1 to 15 and generate a plot of the CS value against the cumulative error level. What do you observe? </font> [4 marks]

In [ ]:
### YOUR CODE HERE
import matplotlib.pyplot as plt

model.eval()
cs_values = []
error_levels = range(1, 16)
for error_level in error_levels:
    total_cs = 0.0
    with torch.no_grad():
        for data, labels in test_loader:
            data = data.float()
            labels = labels.float()
            predictions = model(data)
            cs=(torch.abs(predictions - labels) <= error_level).float().mean().item()
            total_cs += cs * data.size(0)
    cs_value = total_cs / len(test_loader.dataset)
    cs_values.append(cs_value)


plt.plot(error_levels, cs_values, marker='o', linestyle='-', color='b')
plt.title("CS Value vs Cumulative Error Level")
plt.xlabel("Cumulative Error Level")
plt.ylabel("CS Value")
plt.grid(True)
plt.xticks(error_levels)
plt.show()

# 2. PLS and Regression tree

We will compare the results of the linear regression above with Partial Least Squares and Regression Tree methods.

For this we will use scikit-learn [PLSRegression](https://scikit-learn.org/stable/modules/generated/sklearn.cross_decomposition.PLSRegression.html) and [DecisionTreeRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html).

Task 4: Compute the MAE and CS values (with cumulative error level of 5) for both partial least square regression model and the regression tree model. <font color = 'red'> Compare with the previous method </font> [4 marks]

In [ ]:
train_data, train_labels = train['features'], train['labels']
test_data, test_labels = test['features'], test['labels']

In [ ]:
### YOUR CODE HERE
# PLS Regression
from sklearn.metrics import mean_absolute_error
pls = PLSRegression()
pls.fit(train_data, train_labels)

test_predictions_pls = pls.predict(test_data)

mae_pls = mean_absolute_error(test_labels, test_predictions_pls)
cs_pls = np.mean(np.abs(test_predictions_pls - test_labels) <= 5)

print(f"PLS Regression MAE: {mae_pls}")
print(f"PLS Regression CS (Error <= 5): {cs_pls}")

In [ ]:
### YOUR CODE HERE
# Regression Tree
from sklearn.tree import DecisionTreeRegressor
regression_tree = DecisionTreeRegressor()
regression_tree.fit(train_data, train_labels)

test_predictions_tree = regression_tree.predict(test_data)

mae_regression_tree = mean_absolute_error(test_labels, test_predictions_tree)
cs_regression_tree = np.mean(np.abs(test_predictions_tree - test_labels.ravel()) <= 5)

print(f"Regression Tree MAE: {mae_regression_tree}")
print(f"Regression Tree CS (Error <= 5): {cs_regression_tree}")

# 3. SVR

Task 5:  Compute the MAE and CS values (with cumulative error level of 5) for Support Vector Regression. <font color = 'red'> Compare with the previous methods. How do you explain the results in the method comparison? Are the scores expected?</font> [4 marks]

In [ ]:
### YOUR CODE HERE
# SVR
from sklearn.svm import SVR
svr = SVR()
svr.fit(train_data, train_labels.ravel())

svr_predictions = svr.predict(test_data)

mae = mean_absolute_error(test_labels, svr_predictions)
cs = np.mean(np.abs(svr_predictions - test_labels.ravel()) <= 5)

print(f"Support Vector Regression MAE: {mae}")
print(f"Support Vector Regression CS (Error <= 5): {cs}")